In [1]:
!pip install transformers

import os
import pandas as pd
import gensim 
import numpy as np
import keras
import transformers
import tensorflow as tf
from transformers import BertModel, TFBertModel
from gensim.models import KeyedVectors
from gensim.parsing.preprocessing import remove_stopwords
from keras import layers
from keras import optimizers
embedding_size = 300
max_words = 5000
max_review_length = 70
prop_val = 0.2  
data = pd.read_csv('../input/emotion-detection-from-text/tweet_emotions.csv', on_bad_lines='skip')
import sys
print("Python version")
print (sys.version)
print("Version info.")
print (sys.version_info)

Python version
3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 06:08:53) 
[GCC 9.4.0]
Version info.
sys.version_info(major=3, minor=7, micro=12, releaselevel='final', serial=0)


In [2]:
data = data[data["sentiment"] != "empty"];
data = data[data["sentiment"] != "neutral"];
data = data[data["sentiment"] != "surprise"];
data = data[data["sentiment"] != "fun"];
data = data[data["sentiment"] != "hate"];
data = data[data["sentiment"] != "boredom"];
data = data[data["sentiment"] != "relief"];
data = data[data["sentiment"] != "anger"];

In [3]:

print(data.sentiment.unique().tolist())

['sadness', 'enthusiasm', 'worry', 'love', 'happiness']


In [4]:
totalRows = len(data.tweet_id)
print(totalRows)
from transformers import BertTokenizer, TFBertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
transformer_model = TFBertModel.from_pretrained("bert-base-uncased")

23434


Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

2022-05-10 22:49:54.862403: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-10 22:49:54.863502: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-10 22:49:54.864153: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-10 22:49:54.865057: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [5]:
with tf.device("/device:GPU:0"):    


    import re
    sentiment = []
    tweet = []


    for i in range(totalRows):
        tweet_bert = data.iloc[i, 2]
        tweet_bert = remove_stopwords(tweet_bert)
        tweet_bert = re.sub('@[\w]+','',tweet_bert)
        tweet_bert = re.sub('=+','',tweet_bert)
        tweet_bert = re.sub('[^a-zA-Z0-9 \n\.]', '', tweet_bert)
        tweet_bert = str("[CLS] " + tweet_bert + "[SEP]" ) 
        tweet.append(tweet_bert)
        sentiment.append(data.iloc[i, 1])
    print(tweet[0])



  
      
        
        


    #for i in range(totalRows):
      #tokens = tokenizer.tokenize(tweet[i])  

np.random.seed(2)
np.random.shuffle(tweet)
np.random.seed(2)
np.random.shuffle(sentiment)





[CLS] Layin n bed headache ughhhh...waitin call...[SEP]


In [6]:
from tensorflow.keras.utils import to_categorical

from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
sentiment = encoder.fit_transform(sentiment)
print(sentiment)

[[0 0 0 0 1]
 [0 0 0 0 1]
 [0 0 1 0 0]
 ...
 [0 0 1 0 0]
 [0 0 1 0 0]
 [0 0 0 0 1]]


In [7]:
def getModel():

  from tensorflow.keras import regularizers

  input_ids = tf.keras.layers.Input(shape=(max_review_length,), name='input_token', dtype='int32')
  attention_mask = tf.keras.layers.Input(shape=(max_review_length,), name='attention_token', dtype='int32')
  token_type_ids = tf.keras.layers.Input(shape=(max_review_length,), name='token_type_ids_token', dtype='int32')

  x = transformer_model(input_ids, attention_mask)[0]
  x = tf.keras.layers.Flatten()(x)
  x = tf.keras.layers.Dropout(0.8)(x)
  x = tf.keras.layers.Dense(64, bias_regularizer=regularizers.L2(1e-1))(x)
  x = tf.keras.layers.Dropout(0.8)(x)
  x = tf.keras.layers.Dense(5,activation='softmax')(x)


  model = tf.keras.Model(inputs=[input_ids, attention_mask, token_type_ids], outputs = x)   

  model.summary()
  import tensorflow_addons as tfa
  model.compile(loss='CategoricalCrossentropy', optimizer=tf.keras.optimizers.Adam(lr=0.00005), metrics=['acc'])
  my_callbacks = [tf.keras.callbacks.EarlyStopping(patience=4)]
  return model



In [8]:

x_train = tweet[100:]
from transformers import BertTokenizer, TFBertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
!pip install tensorflow-addons
import tensorflow_addons as tfa

with tf.device('/device:GPU:0'):
  len_val = int(len(tweet) * prop_val)
  x_train = tweet[len_val:]
  y_train = np.array(sentiment[len_val:])

  x_val = tweet[:len_val]
  y_val = np.array(sentiment[:len_val])

  x_train_tokenised = tokenizer(x_train, padding='max_length', max_length=max_review_length, truncation=True, return_tensors='tf')
  x_val_tokenised = tokenizer(x_val, padding='max_length', max_length=max_review_length, truncation=True, return_tensors='tf')

 
  x_train_inputs_ids = x_train_tokenised['input_ids']
  x_train_mask = x_train_tokenised['attention_mask']
  x_train_tokens = x_train_tokenised['token_type_ids']

  x_val_inputs_ids = x_val_tokenised['input_ids']
  x_val_mask = x_val_tokenised['attention_mask']
  x_val_tokens = x_val_tokenised['token_type_ids']
    

  network = getModel()

  hist = network.fit((x_train_inputs_ids, x_train_mask, x_train_tokens), 
      y_train, epochs=4, batch_size=64, 
      validation_data=((x_val_inputs_ids, x_val_mask, x_val_tokens),y_val))
        
  mae_hist = hist.history['acc']
  




Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_token (InputLayer)        [(None, 70)]         0                                            
__________________________________________________________________________________________________
attention_token (InputLayer)    [(None, 70)]         0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     TFBaseModelOutputWit 109482240   input_token[0][0]                
                                                                 attention_token[0][0]            
__________________________________________________________________________________________________
flatten (Flatten)               (None, 53760)        0           tf_bert_model[0][0]          

/opt/conda/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
2022-05-10 22:50:33.275838: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


293/293 [==============================] - 174s 548ms/step - loss: 2.6683 - acc: 0.3184 - val_loss: 1.2438 - val_acc: 0.4987
Epoch 2/4
293/293 [==============================] - 158s 538ms/step - loss: 1.8892 - acc: 0.4023 - val_loss: 1.2324 - val_acc: 0.5158
Epoch 3/4
293/293 [==============================] - 157s 537ms/step - loss: 1.5892 - acc: 0.4498 - val_loss: 1.1837 - val_acc: 0.5230
Epoch 4/4
293/293 [==============================] - 157s 537ms/step - loss: 1.3731 - acc: 0.5031 - val_loss: 1.2946 - val_acc: 0.5019


In [9]:
network.save("bertModelReduced")
import sklearn
class_names = ['sadness', 'worry', 'surprise', 'love', 'fun', 'hate', 'happiness']
y_predict = network.predict((x_val_inputs_ids, x_val_mask, x_val_tokens))
print(encoder.inverse_transform(y_predict))

2022-05-10 23:02:24.831157: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


['worry' 'worry' 'happiness' ... 'happiness' 'worry' 'worry']


In [10]:
def process(text):
        import re
        
        from gensim.parsing.preprocessing import remove_stopwords, preprocess_string
        text = remove_stopwords(text)
        text = re.sub('@[\w]+','',text)
        text = re.sub('=+','',text)
        text = re.sub('[^a-zA-Z0-9 \n\.]', '', text)
        return text;



In [11]:

import transformers
import re
from transformers import BertTokenizer, TFBertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
text = "i cant stand my hero academia. how do people watch this crap"
text = process(text)
text = str("[CLS] " + text + "[SEP]" ) 
text = tokenizer(text, padding='max_length', max_length=max_review_length, truncation=True, return_tensors='tf')
ids = text['input_ids']
print (ids.shape)
token = text['token_type_ids']
mask =text['attention_mask']

prediction = model.predict((ids, token, mask))
from sklearn import preprocessing

print(encoder.inverse_transform(prediction))

(1, 70)


NameError: name 'model' is not defined